In [2]:
#set up postgress sql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
engine = create_engine('postgresql://@localhost:5432/nfl')
pd.read_sql("SELECT * FROM information_schema.tables LIMIT 3;", engine)




,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,nfl,pg_catalog,pg_statistic,BASE TABLE,None,None,None,None,None,YES,NO,None
1,nfl,pg_catalog,pg_type,BASE TABLE,None,None,None,None,None,YES,NO,None
2,nfl,pg_catalog,pg_authid,BASE TABLE,None,None,None,None,None,YES,NO,None


In [2]:
%load_ext sql

/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [3]:
%sql postgresql://@localhost:5432/nfl

u'Connected: @nfl'

In [4]:
pwd

u'/Users/egorkharakozov/DSI-projects/CapstoneProject/Part2'

In [5]:
#data is saved in csv files

ls ../../../ProjectData/NFL/nfl_00-15/csv/

BLOCK.csv*    GAME.csv*     OLINE.csv*    PUNT.csv*     TD.csv*
CONV.csv*     INJURY.csv*   PASS.csv*     REDZONE.csv*  TEAM.csv*
DEFENSE.csv*  INTERCPT.csv* PBP.csv*      RUSH.csv*
DRIVE.csv*    KICKER.csv*   PENALTY.csv*  SACK.csv*
FGXP.csv*     KOFF.csv*     PLAY.csv*     SAFETY.csv*
FUMBLE.csv*   OFFENSE.csv*  PLAYER.csv*   TACKLE.csv*


In [6]:
import re

In [7]:
import commands
import re
file_names = "".join(commands.getstatusoutput(
    'ls ../../../ProjectData/NFL/nfl_00-15/csv/')[1]).replace(".csv", "").split("\n")
file_names

['BLOCK',
 'CONV',
 'DEFENSE',
 'DRIVE',
 'FGXP',
 'FUMBLE',
 'GAME',
 'INJURY',
 'INTERCPT',
 'KICKER',
 'KOFF',
 'OFFENSE',
 'OLINE',
 'PASS',
 'PBP',
 'PENALTY',
 'PLAY',
 'PLAYER',
 'PUNT',
 'REDZONE',
 'RUSH',
 'SACK',
 'SAFETY',
 'TACKLE',
 'TD',
 'TEAM']

In [8]:
def load_to_sql(name):
    df = pd.read_csv('../../../ProjectData/NFL/nfl_00-15/csv/{}.csv'.format(name), 
                     encoding='latin1')
    df.to_sql(name, engine, flavor='postgres', if_exists='replace')
    print "done", name

In [147]:
for name in file_names:
    load_to_sql(name.lower())

done block
done conv
done defense
done drive
done fgxp
done fumble
done game
done injury
done intercpt
done kicker
done koff
done offense
done oline
done pass
done pbp
done penalty
done play
done player
done punt
done redzone
done rush
done sack
done safety
done tackle
done td
done team


#### In this notebook I load the data to an SQL database and use python to generate query language to run in SQL to calculate running averages.

In [13]:
###'2: player level avg stats for past few games vs all opps

pcols = ["pa","pc","py","ints", "tdp","fuml", "ra", "sra", "ry", "tdr", "trg", "rec", "recy", "tdrec", "snp"]
paverages = [1,2,3,4,5,8,12,16]
#sql_printer_p(pcols, paverages)

def sql_printer_p(statlist, numlist):
    string = """

"""
    for word in statlist:
        temp = ""
        for n in numlist:
            temp+= """\
avg(case when o.{} is null then 0 else o.{} end) over(partition by o.player order by o.gid asc rows {} preceding)\
as avg_p_vs_all_{},\n""".format(word, word, n-1, word +str(n))

            #avg(case when r.{} is null then 0 else r.{} end) 

        string +=temp

    return  string
    
print sql_printer_p(pcols, paverages)[:1000]




avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 0 preceding)as avg_p_vs_all_pa1,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 1 preceding)as avg_p_vs_all_pa2,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 2 preceding)as avg_p_vs_all_pa3,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 3 preceding)as avg_p_vs_all_pa4,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 4 preceding)as avg_p_vs_all_pa5,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 7 preceding)as avg_p_vs_all_pa8,
avg(case when o.pa is null then 0 else o.pa end) over(partition by o.player order by o.gid asc rows 11 preceding)as avg_p_vs_all_pa12,
avg(case when o.pa is null then 0 else o.pa end) over(partition b

In [24]:
#'3: player level redzone avg stats averages vs all opps'

rzcols = ["pa","pc","py","ints","ra", "sra", "ry", "trg", "rec", "recy"] #stats to pull from rz table
rzaverages = [1, 2,3,4,5,9,12,16]   #number of games for running average
#sql_printer_rz(rzcols, rzaverages)

def sql_printer_rz(statlist, numlist):
    string = """

"""
    for word in statlist:
        temp = ""
        for n in numlist:
            temp+= """avg(case when r.{} is null then 0 else r.{} end) 
over(partition by o.player order by o.gid asc rows {} preceding) as avg_rz_p_vs_all{},\n""".format(word, word, n-1, word +str(n))

        string +=temp

    return string

    
#sql_printer_rz(rzcols, rzaverages)

In [25]:
### 4 calculated columns for offensive players

calced_cols_o = [
"avg(o.pc/nullif(o.pa+o.pc,0))", "avg(o.py/nullif(o.pa,0))", "avg(o.tdp/nullif(o.pa,0))", "avg(o.sra/nullif(o.ra,0))",
"avg(o.tdr / nullif(o.ra,0))",  "avg(o.rec/nullif(o.trg,0))", "avg(o.recy/nullif(o.trg,0))", 
"avg(o.ra+nullif(o.trg,0))", "avg(o.ra+o.rec)", "avg(o.tdrec/nullif(o.rec,0))",
"avg((o.tdr+o.tdrec+o.tdp)/nullif(o.snp,0))", "avg(o.fp3/nullif(o.snp,0))", "avg(o.ry/nullif(o.ra,0))"
]

nameso = ["pass_acc","yds_per_att","td_per_pa", "succ_rush", "td_per_ra", "targ_conv", "yds_per_targ", "rushtargs",
        "touches", "td_per_rec", "td_per_snap", "fp_per_snap", "yds_per_rush"]

cco_avgs = [1,2,3,4,5,8,16]
#calced_o(calced_cols_o, cco_avgs, nameso)

def calced_o(statlist, numlist, namelist):
    string = """

"""
    for index, word in enumerate(statlist):
        temp="""

"""
        for n in numlist:
            temp+= """\
case when (
{} over(partition by o.player order by o.gid asc rows {} preceding)) 
is null then 0 else 
({} over(partition by o.player order by o.gid asc rows {} preceding)) end 
as avg_p_vs_all_{},\n""".format(word, n-1, word, n-1, namelist[index] + str(n-1))

        string +=temp
    return string

# calced_o(calced_cols_o, cco_avgs, nameso)

In [26]:
### 5 calculated columns for offensive players

calced_cols_rz = [
"avg(r.pc/nullif(r.pa+r.pc,0))", "avg(r.py/nullif(r.pa,0))", 
"avg(r.sra/nullif(r.ra,0))",  "avg(r.rec/nullif(r.trg,0))",  "avg(r.recy/nullif(r.trg,0))", 
"avg(r.ra + nullif(r.trg,0))", "avg(r.ra + r.rec)"
]

namesrz = ["pass_acc","yds_per_att", "succ_rush", "targ_conv", "yds_per_targ", "rushtargs",
        "touches"]

ccrz_avgs = [1, 2,3,4,5,8,10,12,16]

#calced_rz(calced_cols_rz, ccrz_avgs, namesrz)

def calced_rz(statlist, numlist, namelist):
    string = """

"""
    for index, word in enumerate(statlist):
        temp="""
"""
        for n in numlist:
            temp+= """\
case when (
{} over(partition by o.player order by o.gid asc rows {} preceding))
is null then 0 else
({} over(partition by o.player order by o.gid asc rows {} preceding)) end
as avg_p_rz_vs_all_cc_{},\n""".format(word, n-1, word, n-1, namesrz[index] + str(n-1))

        string +=temp
    return string
# calced_rz(calced_cols_rz, ccrz_avgs, namesrz)

In [27]:
### 6 calculated columns for teams

calced_cols_tm = [
"avg(t.pc/nullif(t.pa+t.pc,0))", "avg(t.py/nullif(t.pa,0))", 
"avg(t.sra/nullif(t.ra,0))", "avg(t.ry/nullif(t.ra,0))", "avg(t.td/ nullif(t.snpo,0))",
    "avg(t.top/ nullif(t.snpo,0))"
]

namestm = ["pass_acc","yds_per_att", "succ_rush", "yds_per_rush", "td_per_snap", "time_per_snap" ]

cctm_avgs = [2,3,4,5,6,7,8,12,16]

#calced_tm(calced_cols_tm, cctm_avgs, namestm)

def calced_tm(statlist, numlist, namelist):
    string = """

"""
    for index, word in enumerate(statlist):
        temp="""
"""
        for n in numlist:
            temp+= """\
case when ({} over(partition by o.player order by o.gid asc rows {} preceding))
is null then 0 else 
({} over(partition by o.player order by o.gid asc rows {} preceding)) end
as avg_tm_vs_all_{},\n""".format(word, n-1, word, n-1, namestm[index] +str(n-1))


        string +=temp
    return string

# calced_tm(calced_cols_tm, cctm_avgs, namestm)

In [28]:
#team level stats for players team and the opposing team for past few games (team offense and opponent defense strength)

tm_avg_cols = ['pts', 'rfd', 'pfd', 'ifd', 'ry', 'ra', 'py', 'pa', 'pc', 
          'sk', 'ints', 'fum', 'top', 
          'td', 'tdr', 'tdp', 'tdt', 'rza', 'rzc', 'bry', 'bpy', 'srp', 's1rp', 
          's2rp', 's3rp', 'spp', 's1pp', 's2pp', 's3pp', 'lea', 'ley', 'lta', 'lty', 'lga', 'lgy', 
          'mda', 'mdy', 'rga', 'rgy', 'rta', 'rty', 'rea', 'rey', 'r1a', 'r1y', 'r2a', 'r2y', 'r3a', 
          'r3y', 'qba', 'qby', 'sla', 'sly', 'sma', 'smy', 'sra', 'sry', 'dla', 'dly', 'dma', 'dmy', 
          'dra', 'dry', 'wr1a', 'wr1y', 'wr3a', 'wr3y', 'tea', 'tey', 'rba', 'rby', 'sga', 'sgy', 
          'p1a', 'p1y', 'p2a', 'p2y', 'p3a', 'p3y', 'spc', 'mpc', 'lpc', 
          'lcra', 'lcry', 'lcpa', 'lcpy', 'rzra', 'rzry', 'rzpa', 'rzpy', 'sky', 'lbs', 'dbs', 'sfpy', 
          'drv', 'lnr', 'lnp', 'lbr', 'lbp', 'dbr', 'dbp', 'nha', 's3a', 's3c', 
          'l3a', 'l3c', 'stf', 'dp', 'pfn', 'snpo', 'snpd'] 



#stats to pull from team table
teamaverages2 = [1,3,6,11]  #number of games for running average
#sql_printer_team(tmcols, teamaverages)


def sql_printer_team_avg(statlist, numlist):
    string = """


"""
    for word in statlist:
        temp = ""
        for n in numlist:
            temp+= """
avg(case when t.{} is null then 0 else t.{} end) 
over(partition by t.tname order by t.gid asc rows {} preceding) as tm_O_{},

avg(case when tt.{} is null then 0 else tt.{} end) 
over(partition by t.tname order by t.gid asc rows {} preceding) as tm_D_{},\n""".format(word, word, n-1, 
                                                                                        word +str(n), word, 
                                                                                        word, n-1, word +str(n))

        string +=temp
    string = "select t.gid, t.tname,\n" + string[:-2] + """
from team t, team tt
where t.gid = tt.gid
and t.tname <> tt.tname
"""
    return string

    
# sql_printer_team_avg(tm_avg_cols, teamaverages2)

In [29]:
sql = sql_printer_team_avg(tm_avg_cols, teamaverages2)

df = pd.read_sql(sql, engine)
len(df)

8512

In [30]:
df.to_sql("team_avg", engine, flavor='postgres', if_exists='replace')

In [31]:
#HEAD
head = """


--'this query is the general query to pull all of the data from underlying tables

select

-- 'first section is target and general player /game info, fanduel fantasy pts are the target

o.gid,
o.player,
g.wk,

"""
#TABLE1: weather, injuries, vegas, player specs
table1="""

o.fp3 as target, --'need to figure out the shift in sql'
o.team,
case when o.team = g.h then g.v else g.h end as opponent,

p.fname,
p.lname,
p.pos1,

p.dcp, -- 'depth chart position CAT 

--'injury statuses, practice participation and game status, all CAT
i.pstat, --'CAT'
i.gstat, -- 'CAT'
i.details, -- 'CAT'

--'game info, weather, location
g.seas,
g.stad, --'CAT'
case when o.team = g.h then 1 else 0 end as home,
case when g.cond in ('Dome', 'Closed Roof') then 'indoor'
when g.temp > 85 then 'hot' when g.temp between 84 and 50 then 'mild' else 'cold' end as temp
, --'CAT'

g.surf, -- 'CAT'
g.cond, --'CAT'

--'vegas odds info, first the over under (total pts per game), second the implied team score
g.ou,
case when o.team = g.h then g.ou/2 +g.sprv/2 else g.ou/2 -g.sprv/2 end as imp_score,

--'player specific, static attributes, height, weight, draft status etc.
p.height,
p.weight,
g.seas - p.yob as age_in_season,
p.forty,
p.bench,
p.vertical,
p.broad,
p.shuttle,
p.cone,
p.arm,
p.hand,
p.dpos, -- draft pos
g.seas-p.start as yrs_exp,
"""


#TABLE2: player level avg stats for past few games vs all opps

pcols = [ "pa","pc","py","ints", "tdp","fuml", "ra", "sra", "ry", "tdr", "trg", "rec", "recy", "tdrec", "snp"]
paverages = [1,2,3,4,5,8,12,16]

table2 = sql_printer_p(pcols, paverages)

#TABLE3: player level avg redzone stats for past few games vs all opps

rzcols = ["pa","pc","py","ints","ra", "sra", "ry", "trg", "rec", "recy"] #stats to pull from rz table
rzaverages = [1,2,3,4,5,9,12,16]   #number of games for running average

table3 = sql_printer_rz(rzcols, rzaverages)



#TABLE4: derived percentages player level vs all opps

calced_cols_o = [
"avg(o.pc/nullif(o.pa+o.pc,0))", "avg(o.py/nullif(o.pa,0))", "avg(o.tdp/nullif(o.pa,0))", "avg(o.sra/nullif(o.ra,0))",
"avg(o.tdr / nullif(o.ra,0))",  "avg(o.rec/nullif(o.trg,0))", "avg(o.recy/nullif(o.trg,0))", 
"avg(o.ra+nullif(o.trg,0))", "avg(o.ra+o.rec)", "avg(o.tdrec/nullif(o.rec,0))",
"avg((o.tdr+o.tdrec+o.tdp)/nullif(o.snp,0))", "avg(o.fp3/nullif(o.snp,0))", "avg(o.ry/nullif(o.ra,0))"
]

nameso = ["pass_acc","yds_per_att","td_per_pa", "succ_rush", "td_per_ra", "targ_conv", "yds_per_targ", "rushtargs",
        "touches", "td_per_rec", "td_per_snap", "fp_per_snap", "yds_per_rush"]

cco_avgs = [1,2,3,4,5,8,16]


table4 = calced_o(calced_cols_o, cco_avgs, nameso)

#TABLE5: derived percentages player level vs all opps in rz

calced_cols_rz = [
"avg(r.pc/nullif(r.pa+r.pc,0))", "avg(r.py/nullif(r.pa,0))", 
"avg(r.sra/nullif(r.ra,0))",  "avg(r.rec/nullif(r.trg,0))",  "avg(r.recy/nullif(r.trg,0))", 
"avg(r.ra + nullif(r.trg,0))", "avg(r.ra + r.rec)"
]

namesrz = ["pass_acc","yds_per_att", "succ_rush", "targ_conv", "yds_per_targ", "rushtargs",
        "touches"]

ccrz_avgs = [1,2,3,4,5,8,10,12,16]



table5 = calced_rz(calced_cols_rz, ccrz_avgs, namesrz)


#TABLE6: calculated columns for teams

calced_cols_tm = [
"avg(t.pc/nullif(t.pa+t.pc,0))", "avg(t.py/nullif(t.pa,0))", 
"avg(t.sra/nullif(t.ra,0))", "avg(t.ry/nullif(t.ra,0))", "avg(t.td/ nullif(t.snpo,0))",
    "avg(t.top/ nullif(t.snpo,0))"
]

namestm = ["pass_acc","yds_per_att", "succ_rush", "yds_per_rush", "td_per_snap", "time_per_snap" ]

cctm_avgs = [1,2,3,4,5,6,7,8,12,16]


table6 = calced_tm(calced_cols_tm, cctm_avgs, namestm)


#the tail query includes all fields from the team avg table

regtail = """

ta.*

from offense o 
left outer join injury i on (i.gid = o.gid and i.player = o.player)
left outer join redzone r on (r.gid = o.gid and r.player = o.player)
left outer join team t on (t.gid = o.gid and t.tname = o.team) --'players team
left outer join team ot on (ot.gid = o.gid and ot.tname <> o.team)
left outer join team_avg ta on (ta.gid = o.gid and ta.tname = o.team),
player p, 
game g 


where p.player = o.player
and g.gid = o.gid

"""

In [32]:
sql = head + table1 + table2 +table3 + table4 + table5 + table6 + regtail


df = pd.read_sql(sql, engine)

len(df)

88592

In [33]:
np.shape(df)

(88592, 1339)

In [35]:
df.to_csv("~/ProjectData/NFL/all_data_10_13_v2.csv")